| [05_spider/01_爬虫介绍.ipynb](https://github.com/shibing624/python-tutorial/blob/master/05_spider/01_爬虫介绍.ipynb)  | Python网络爬虫介绍  |[Open In Colab](https://colab.research.google.com/github/shibing624/python-tutorial/blob/master/05_spider/01_爬虫介绍.ipynb) |

# 爬虫介绍

## 网络爬虫的概念

网络爬虫（web crawler），以前经常称之为网络蜘蛛（spider），是按照一定的规则自动浏览万维网并获取信息的机器人程序（或脚本），曾经被广泛的应用于互联网搜索引擎。使用过互联网和浏览器的人都知道，网页中除了供用户阅读的文字信息之外，还包含一些超链接。网络爬虫系统正是通过网页中的超链接信息不断获得网络上的其它页面。正因如此，网络数据采集的过程就像一个爬虫或者蜘蛛在网络上漫游，所以才被形象的称为网络爬虫或者网络蜘蛛。

## 爬虫的应用领域

对于大多数的公司而言，及时的获取行业相关数据是企业生存的重要环节之一，然而大部分企业在行业数据方面的匮乏是其与生俱来的短板，合理的利用爬虫来获取数据并从中提取出有商业价值的信息是至关重要的。

**网络数据采集是Python最擅长的领域之一。**

当我们在浏览器中输入一个url后回车，后台会发生什么？比如说你输入[https://www.baidu.com/](https://www.baidu.com/)，你就会看到百度首页。

简单来说这段过程发生了以下四个步骤：

1. 查找域名对应的IP地址
2. 向IP对应的服务器发送请求
3. 服务器响应请求，发回网页内容
4. 浏览器解析网页内容

网络爬虫要做的，简单来说，就是实现浏览器的功能。通过指定url，直接返回给用户所需要的数据，而不需要一步步人工去操纵浏览器获取。


网络爬虫主要分3个大的版块：**抓取**，**解析**，**存储** 

# 抓取

## 豆瓣爬虫
从豆瓣上爬取Top250电影名称：

In [8]:
import random
import time

import requests
from bs4 import BeautifulSoup

for page in range(2):
    resp = requests.get(
        url=f'https://movie.douban.com/top250?start={25 * page}',
        headers={'User-Agent': 'BaiduSpider'}
    )
    soup = BeautifulSoup(resp.text, "lxml")
    for elem in soup.select('a > span.title:nth-child(1)'):
        print(elem.text)
    time.sleep(random.random() * 5)

肖申克的救赎
霸王别姬
阿甘正传
这个杀手不太冷
泰坦尼克号
美丽人生
千与千寻
辛德勒的名单
盗梦空间
忠犬八公的故事
星际穿越
楚门的世界
海上钢琴师
三傻大闹宝莱坞
机器人总动员
放牛班的春天
无间道
疯狂动物城
大话西游之大圣娶亲
熔炉
教父
当幸福来敲门
龙猫
怦然心动
控方证人
触不可及
末代皇帝
蝙蝠侠：黑暗骑士
寻梦环游记
活着
指环王3：王者无敌
哈利·波特与魔法石
乱世佳人
何以为家
素媛
飞屋环游记
摔跤吧！爸爸
十二怒汉
哈尔的移动城堡
少年派的奇幻漂流
我不是药神
鬼子来了
大话西游之月光宝盒
天空之城
天堂电影院
闻香识女人
指环王2：双塔奇兵
罗马假日
猫鼠游戏
辩护人



## 搜狐体育爬虫

从“搜狐体育”上获取NBA新闻标题和链接的爬虫：

In [7]:
import re
from collections import deque
from urllib.parse import urljoin

import requests

LI_A_PATTERN = re.compile(r'<li class="item">.*?</li>')
A_TEXT_PATTERN = re.compile(r'<a\s+[^>]*?>(.*?)</a>')
A_HREF_PATTERN = re.compile(r'<a\s+[^>]*?href="(.*?)"\s*[^>]*?>')


def decode_page(page_bytes, charsets):
    """通过指定的字符集对页面进行解码"""
    for charset in charsets:
        try:
            return page_bytes.decode(charset)
        except UnicodeDecodeError:
            pass


def get_matched_parts(content_string, pattern):
    """从字符串中提取所有跟正则表达式匹配的内容"""
    return pattern.findall(content_string, re.I) \
        if content_string else []


def get_matched_part(content_string, pattern, group_no=1):
    """从字符串中提取跟正则表达式匹配的内容"""
    match = pattern.search(content_string)
    if match:
        return match.group(group_no)


def get_page_html(seed_url, *, charsets=('utf-8', )):
    """获取页面的HTML代码"""
    resp = requests.get(seed_url)
    if resp.status_code == 200:
        return decode_page(resp.content, charsets)


def repair_incorrect_href(current_url, href):
    """修正获取的href属性"""
    if href.startswith('//'):
        href = urljoin('http://', href)
    elif href.startswith('/'):
        href = urljoin(current_url, href)
    return href if href.startswith('http') else ''


def start_crawl(seed_url, pattern, *, max_depth=-1):
    """开始爬取数据"""
    new_urls, visited_urls = deque(), set()
    new_urls.append((seed_url, 0))
    while new_urls:
        current_url, depth = new_urls.popleft()
        if depth != max_depth:
            page_html = get_page_html(current_url, charsets=('utf-8', 'gbk'))
            contents = get_matched_parts(page_html, pattern)
            for content in contents:
                text = get_matched_part(content, A_TEXT_PATTERN)
                href = get_matched_part(content, A_HREF_PATTERN)
                if href:
                    href = repair_incorrect_href(current_url, href)
                print(text, href)
                if href and href not in visited_urls:
                    new_urls.append((href, depth + 1))


def main():
    """主函数"""
    start_crawl(
        seed_url='http://sports.sohu.com/nba_a.shtml',
        pattern=LI_A_PATTERN,
        max_depth=2
    )


if __name__ == '__main__':
    main()

直播 http://data.sports.sohu.com/nba/nba_schedule_by_day.html
赛程 http://data.sports.sohu.com/nba/nba_schedule_by_month.php?
季后赛 http://sports.sohu.com/s/nba/playoffs
排名 http://data.sports.sohu.com/nba/nba_teams_rank.html
球队 http://data.sports.sohu.com/nba/nba_teams.html
球员 http://data.sports.sohu.com/nba/nba_players.html
数据 http://data.sports.sohu.com/nba/index.html
深度 http://www.sohu.com/subject/315837
CBA http://sports.sohu.com/s/cba
男篮 http://sports.sohu.com/s/tcb
女篮 http://cbachina.sports.sohu.com/s2011/tcbw/
NBL http://sports.sohu.com/nbl/
保罗：希望和哈登沟通过各自角色 休城时光有些灰暗 https://www.sohu.com/a/487971684_458722?scm=1004.728313668755980288.0.0.0
深度：三巨头如虎添翼 悍将加盟成篮网新季强心针 https://www.sohu.com/a/487845721_458722?scm=1004.728313668755980288.0.0.0
全明星阵容最强球队：新赛季湖人领衔 11年绿军第2 https://www.sohu.com/a/488008799_458722?scm=1004.728313668755980288.0.0.0
东契奇：基德是位优秀教练 新赛季目标是竞争总冠军 https://www.sohu.com/a/488005318_458722?scm=1004.728313668755980288.0.0.0
豪斯主动示好火箭盼长留 因号码危机卷入交易传闻 https://www.sohu.com/a/48800192


通过上面的例子，我们对爬虫已经有了一个感性的认识，在编写爬虫时有以下一些注意事项：

1. 上面的代码使用了`requests`三方库来获取网络资源，这是一个非常优质的三方库，关于它的用法可以参考它的[官方文档](https://requests.readthedocs.io/zh_CN/latest/)。

2. 上面的代码中使用了双端队列（`deque`）来保存待爬取的URL。双端队列相当于是使用链式存储结构的`list`，在双端队列的头尾添加和删除元素性能都比较好，刚好可以用来构造一个FIFO（先进先出）的队列结构。

3. 处理相对路径。有的时候我们从页面中获取的链接不是一个完整的绝对链接而是一个相对链接，这种情况下需要将其与URL前缀进行拼接（`urllib.parse`中的`urljoin()`函数可以完成此项操作）。

4. 设置代理服务。有些网站会限制访问的区域（例如美国的Netflix屏蔽了很多国家的访问），有些爬虫需要隐藏自己的身份，在这种情况下可以设置使用代理服务器，代理服务器有免费的服务器和付费的商业服务器，但后者稳定性和可用性都更好，强烈建议在商业项目中使用付费的商业代理服务器。如果使用`requests`三方库，可以在请求方法中添加`proxies`参数来指定代理服务器；如果使用标准库，可以通过修改`urllib.request`中的`ProxyHandler`来为请求设置代理服务器。

5. 限制下载速度。如果我们的爬虫获取网页的速度过快，可能就会面临被封禁或者产生“损害动产”的风险（这个可能会导致吃官司且败诉），可以在两次获取页面数据之间添加延时从而对爬虫进行限速。

6. 避免爬虫陷阱。有些网站会动态生成页面内容，这会导致产生无限多的页面（例如在线万年历通常会有无穷无尽的链接）。可以通过记录到达当前页面经过了多少个链接（链接深度）来解决该问题，当达到事先设定的最大深度时，爬虫就不再像队列中添加该网页中的链接了。

7. 避开蜜罐链接。网站上的有些链接是浏览器中不可见的，这种链接通常是故意诱使爬虫去访问的蜜罐，一旦访问了这些链接，服务器就会判定请求是来自于爬虫的，这样可能会导致被服务器封禁IP地址。如何避开这些蜜罐链接我们在后面为大家进行讲解。

本节完。